In [22]:
"""
    v2 文本特征挖掘
    tf-idf
    
    提取字段的个数 count()
    唯一值的个数 
    LightGBM = 0.72
    XGBoost = 0.68
    
    0.3 * lgb + 0.7 xgb
    mrege = 0.688 反而下降了
    
"""

import pandas as pd
from pathlib import Path
import pickle


def get_data(filename):
    result = []
    chunk_index = 0
    for df in pd.read_csv(open(filename, 'r'), chunksize = 1000000):
        result.append(df)
        chunk_index += 1
    results = pd.concat(result, ignore_index = True, axis = 0)
    return results




In [23]:
if Path('./datasets/train.pkl').exists():
    with open('./datasets/train.pkl', 'rb') as f:
        train = pickle.load(f)
else:
    train = get_data('./datasets/security_train.csv')
    train.to_pickle('./datasets/train.pkl')

    
if Path('./datasets/test.pkl').exists():
    with open('./datasets/test.pkl', 'rb') as f:
        test = pickle.load(f)
else:
    test = get_data('./datasets/security_test.csv')
    train.to_pickle('./datasets/test.pkl')


In [24]:
def get_apis(df):
    group_fileid = df.groupby('file_id')
    file_api = {}
    for file_id, file_group in group_fileid:
        result = file_group.sort_values(['tid', 'index'], ascending=True)
        api_sequence  = ' '.join(result['api'])
        file_api[file_id] = api_sequence
    return file_api

In [26]:
%%time
# load train apis
if Path('./datasets/train_apis.pkl').exists():
    with open('./datasets/train_apis.pkl', 'rb') as f:
        train_apis = pickle.load(f)
else:
    # file_id 对应的api调用序列
    train_apis = get_apis(train)
    with open('./datasets/train_apis.pkl', 'wb') as f:
        pickle.dump(train_apis, f)

Wall time: 823 ms


In [27]:
len(train_apis)

13887

In [28]:
%%time
# load test apis

from pathlib import Path
if Path('./datasets/test_apis.pkl').exists():
    with open('./datasets/test_apis.pkl', 'rb') as f:
        test_apis = pickle.load(f)
else:
    # file_id 对应的api调用序列
    test_apis = get_apis(test)
    with open('./datasets/test_apis.pkl', 'wb') as f:
        pickle.dump(test_apis, f)


Wall time: 898 ms


In [29]:
# 构造新特征
def get_feature(df):
    df_file = df.groupby('file_id')
    if 'label' in df.columns:
        df1 = df.drop_duplicates(subset = ['file_id', 'label'], keep='first')
    else:
        df1 = df.drop_duplicates(subset = ['file_id'], keep='first')
    df1 = df1.sort_values('file_id')
    
    # 提取多个特征 统计特征
    features = ['tid', 'index']
    for f in features:
        df1[f + '_count'] = df_file[f].count().values
        df1[f + '_nuinque'] = df_file[f].nunique().values
        df1[f + '_min'] = df_file[f].min().values
        df1[f + '_max'] = df_file[f].max().values
        df1[f + '_median'] = df_file[f].median().values
        df1[f + '_std'] = df_file[f].std().values
    return df1


In [30]:
%%time

from pathlib import Path
if Path('./datasets/df_train.pkl').exists():
    with open('./datasets/df_train.pkl', 'rb') as f:
        df_train = pickle.load(f)
else:
    # file_id 对应的api调用序列
    df_train = get_feature(train)
    with open('./datasets/df_train.pkl', 'wb') as f:
        pickle.dump(df_train, f)

df_train

Wall time: 2.99 ms


,file_id,label,api,tid,index,tid_count,tid_nuinque,tid_min,tid_max,tid_median,tid_std,index_count,index_nuinque,index_min,index_max,index_median,index_std
0,1,5,LdrLoadDll,2488,0,6786,11,2488,2812,2488.0,83.881299,6786,5001,0,5000,1607.5,1510.694221
6786,2,2,GetSystemTimeAsFileTime,2320,0,816,4,2320,2604,2480.0,101.506783,816,204,0,203,101.5,58.925137
7602,3,0,NtAllocateVirtualMemory,2208,0,463,1,2208,2208,2208.0,0.000000,463,463,0,462,231.0,133.800847
8065,4,0,GetSystemTimeAsFileTime,2284,0,2046,9,2284,2980,2340.0,150.460506,2046,1028,0,1027,511.0,295.407885
10111,5,0,SetErrorMode,2500,0,10002,6,2500,2676,2596.0,49.556301,10002,5001,0,5000,2500.0,1443.736493
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89620181,13883,2,GetSystemTimeAsFileTime,100,0,178221,998,100,6568,3392.0,1405.045515,178221,5001,0,5000,47.0,1008.636040
89798402,13884,5,GetSystemTimeAsFileTime,2592,0,1319,2,2592,2748,2592.0,4.295386,1319,1319,0,1318,659.0,380.906813
89799721,13885,0,NtAllocateVirtualMemory,2240,0,1033,3,2240,2744,2240.0,33.152020,1033,1033,0,1032,516.0,298.345717
89800754,13886,1,GetSystemTimeAsFileTime,2324,0,5316,10,2324,2836,2600.0,154.796790,5316,2503,0,2502,1165.5,755.545651


In [31]:
%%time

from pathlib import Path
if Path('./datasets/df_test.pkl').exists():
    with open('./datasets/df_test.pkl', 'rb') as f:
        df_test = pickle.load(f)
else:
    # file_id 对应的api调用序列
    df_test = get_feature(test)
    with open('./datasets/df_test.pkl', 'wb') as f:
        pickle.dump(df_test, f)

df_test


Wall time: 3.02 ms


,file_id,api,tid,index,tid_count,tid_nuinque,tid_min,tid_max,tid_median,tid_std,index_count,index_nuinque,index_min,index_max,index_median,index_std
0,1,RegOpenKeyExA,2332,0,97,4,2332,2568,2544.0,57.218548,97,31,0,30,14.0,9.210466
97,2,RegOpenKeyExA,2472,0,1361,7,2472,2748,2524.0,104.399149,1361,681,0,680,340.0,196.515744
1458,3,GetSystemTimeAsFileTime,2344,0,16,1,2344,2344,2344.0,0.000000,16,16,0,15,7.5,4.760952
1474,4,LdrLoadDll,2452,0,193,3,2452,2584,2452.0,50.951508,193,193,0,192,96.0,55.858452
1667,5,GetSystemTimeAsFileTime,2332,0,803,3,2332,2780,2376.0,201.826813,803,268,0,267,133.0,77.317048
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79277890,12951,NtAllocateVirtualMemory,2644,0,289,6,2644,2980,2776.0,75.402526,289,145,0,144,72.0,41.786414
79278179,12952,NtAllocateVirtualMemory,2264,0,112,1,2264,2264,2264.0,0.000000,112,112,0,111,55.5,32.475632
79278291,12953,LdrLoadDll,2324,0,5095,15,2324,2884,2708.0,196.695730,5095,1464,0,1463,454.0,393.605016
79283386,12954,LdrLoadDll,2424,0,2951,4,2424,2700,2680.0,126.124152,2951,1445,0,1444,555.0,397.358069


In [10]:
# if 'api' in df_train.columns:
#     df_train.drop(['api'], axis = 1, inplace = True)
# if 'api' in df_test.columns:
#     df_test.drop(['api'], axis = 1, inplace = True)

In [13]:
if 'api' not in df_test.columns:
    temp = pd.DataFrame.from_dict(test_apis, orient='index', columns=['api'])
    temp = temp.reset_index().rename(columns={'index': 'file_id'})
    df_test = df_test.merge(temp, on='file_id', how="left")

df_train.to_pickle('./df_train.pkl')
df_test.to_pickle('./df_test.pkl')

,file_id,api,tid,index,tid_count,tid_nuinque,tid_min,tid_max,tid_median,tid_std,index_count,index_nuinque,index_min,index_max,index_median,index_std
0,1,RegOpenKeyExA,2332,0,97,4,2332,2568,2544.0,57.218548,97,31,0,30,14.0,9.210466
97,2,RegOpenKeyExA,2472,0,1361,7,2472,2748,2524.0,104.399149,1361,681,0,680,340.0,196.515744
1458,3,GetSystemTimeAsFileTime,2344,0,16,1,2344,2344,2344.0,0.000000,16,16,0,15,7.5,4.760952
1474,4,LdrLoadDll,2452,0,193,3,2452,2584,2452.0,50.951508,193,193,0,192,96.0,55.858452
1667,5,GetSystemTimeAsFileTime,2332,0,803,3,2332,2780,2376.0,201.826813,803,268,0,267,133.0,77.317048
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79277890,12951,NtAllocateVirtualMemory,2644,0,289,6,2644,2980,2776.0,75.402526,289,145,0,144,72.0,41.786414
79278179,12952,NtAllocateVirtualMemory,2264,0,112,1,2264,2264,2264.0,0.000000,112,112,0,111,55.5,32.475632
79278291,12953,LdrLoadDll,2324,0,5095,15,2324,2884,2708.0,196.695730,5095,1464,0,1463,454.0,393.605016
79283386,12954,LdrLoadDll,2424,0,2951,4,2424,2700,2680.0,126.124152,2951,1445,0,1444,555.0,397.358069


In [15]:
df_all = pd.concat([train, test])

In [21]:
# tf idf
from sklearn.feature_extraction.text import TfidfVectorizer

df_all = pd.concat([train, test])
vec = TfidfVectorizer(ngram_range=(1,3), min_df = 0.1, max_df = 0.8)

if Path('./datasets/df_apis.pkl').exists():
    with open('./datasets/df_apis.pkl', 'rb') as f:
        df_apis = pickle.load(f)
else:
    # file_id 对应的api调用序列
    api_features = vec.fit_transform(df_all['api'])
    df_apis = pd.DataFrame(api_features.toarray(), columns = vec.get_feature_names())
    with open('./datasets/df_apis.pkl', 'wb') as f:
        pickle.dump(df_apis, f)
    df_apis = pd.DataFrame(api_features.toarray(), columns=vec.get_feature_names())


In [18]:
api_features = vec.fit_transform(df_all['api'])
api_features

<179613386x1 sparse matrix of type '<class 'numpy.float64'>'
	with 21408610 stored elements in Compressed Sparse Row format>

In [19]:
df_apis

,ldrgetprocedureaddress
0,0.0
1,1.0
2,1.0
3,1.0
4,1.0
...,...
179613381,0.0
179613382,0.0
179613383,0.0
179613384,0.0


In [20]:
df_train_apis = df_apis[df_train.index < 13886]
df_train_apis

ValueError: Item wrong length 13887 instead of 179613386.

In [ ]:
# df_test_apis = df_apis[df_test.index < 13886]
# df_test_apis

In [ ]:
df_train = df_train.merge(df_train_apis, left_index = True, right_index = True)
df_test = df_test.merge(df_test_apis, left_index = True, right_index = True)

In [ ]:
df_train.to_pickle('./df_train2.pkl')
df_test.to_pickle9('./df_test2.pkl')

In [ ]:
df_train

In [ ]:
# 0.72
import lightgbm as lgb

clf = lgb.LGBMClassifier(
    num_leaves=2**5 - 1, reg_alpha=0.25, reg_lambda= 0.25, objective='multiclass', max_depth=-1, learning_rate=0.005, min_child_samples=3,
    random_state=2021, n_estimators=10000, subsample=1, colsample_bytree=1
)

clf.fit(df_train.drop('label', axis=1), df_train['label'])

# result = ensemble_model(clf, features, labels, test_fea, cate_features)
result = clf.predict_proba(df_test)
result = pd.DataFrame(result, columns = ['prob0', 'prob1','prob2','prob3','prob4','prob5','prob6','prob7'])
result['file_id'] = df_test['file_id'].values
columns = ['file_id', 'prob0', 'prob1','prob2','prob3','prob4','prob5','prob6','prob7']
result.to_csv('v2 lgb nl25 ne10000.csv', index = False, columns = columns)


In [ ]:
# %%time

# # 0.683331

# import xgboost as xgb

# model_xgb = xgb.XGBClassifier(
#     max_depth=9, learning_rate=0.005, n_estimators=2000, 
#     objective='multi:softprob', tree_method='gpu_hist', 
#     subsample=0.8, colsample_bytree=0.8, 
#     min_child_samples=3, eval_metric='logloss', reg_lambda=0.5
# )


# model_xgb = model_xgb.fit(df_train.drop('label', axis=1), df_train['label'])

# # result = ensemble_model(clf, features, labels, test_fea, cate_features)
# result = model_xgb.predict_proba(df_test)
# result = pd.DataFrame(result, columns = ['prob0', 'prob1','prob2','prob3','prob4','prob5','prob6','prob7'])
# result['file_id'] = df_test['file_id'].values
# columns = ['file_id', 'prob0', 'prob1','prob2','prob3','prob4','prob5','prob6','prob7']
# result.to_csv('v1 xgb.csv', index = False, columns = columns)
